In [44]:
# importamos las librerias que vamos a utilizar en el notebook

import torch # libreria principal
import torch.nn as nn # libreria para redes neuronales
import torch.optim as optim # libreria para optimizacion
from torch.utils.data import DataLoader, Dataset # libreria para cargar los datos
import torchvision.transforms as transforms # libreria para transformar los datos
from torchvision import models # libreria para modelos pre-entrenados
from datasets import load_dataset # libreria para cargar los datos

In [45]:
# cargamos el dataset de Tiny ImageNet y lo guardamos en la variable dataset
dataset = load_dataset("zh-plus/tiny-imagenet")

In [46]:
# mostramos la estructura del dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
    valid: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [47]:
# Definir una transformación para el preprocesamiento de imágenes.
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3), # pasar a escala de grises (3 canales)
    transforms.Resize((64, 64)), # redimensionar a 64x64
    transforms.ToTensor(), # convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalizar
])

In [48]:
# Defina su clase de conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, dataset, split, transform=None):
        self.X = dataset[split]['image'] # cargar imágenes
        self.y = dataset[split]['label'] # cargar etiquetas
        self.transform = transform

    # Devuelve el número de muestras en el conjunto de datos.
    def __len__(self):
        return len(self.X)

    # Devuelve una muestra del conjunto de datos en la posición index.
    def __getitem__(self, idx):
        img = self.X[idx]
        label = self.y[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

In [49]:
# Cree instancias de su conjunto de datos personalizado para divisiones de entrenamiento y prueba
train_dataset = CustomDataset(dataset, 'train', transform=transform)
test_dataset = CustomDataset(dataset, 'valid', transform=transform)

# Crear cargadores de datos para cargar los datos.
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [50]:
# miraramos el tamaño de los datos de entrenamiento por batches (1er batch) y el tamaño de las etiquetas 
for batch_idx, (X, y) in enumerate(train_loader):
    print(X.shape)
    print(y.shape)
    break

torch.Size([64, 3, 64, 64])
torch.Size([64])


In [64]:
# Cargar el modelo preentrenado EfficientNet-B0
#from efficientnet_pytorch import EfficientNet
#model = EfficientNet.from_pretrained('efficientnet-b0')


model = models.densenet121(pretrained=True)
model

c:\Users\israe\anaconda3\envs\cuda_user\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\israe\anaconda3\envs\cuda_user\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [65]:
# cambiamos la capa de salida de la red por una capa lineal con 200 neuronas (número de clases)
model.classifier

Linear(in_features=1024, out_features=1000, bias=True)

In [66]:
# cambiar la última capa de salida de la red para que coincida con el número de clases en el conjunto de datos
num_ftrs = model.classifier.in_features # número de características de entrada de la última capa
model.classifier = nn.Linear(num_ftrs, 200) # cambiamos a 200 clases de salida

In [67]:
# miramos la nueva arquitectura de la red neuronal convolucional pre-entrenada (ResNet50)
model.classifier

Linear(in_features=1024, out_features=200, bias=True)

In [68]:
# Define a loss function and an optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # precision de un batch 17%
#optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001) # precision de un batch 19.26%
#optimizer = optim.Adamax(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001) # precision de un batch 20.64%

# Train the model on your dataset
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()
        X = model(X)
        loss = criterion(X, y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            X = model(X)
            _, predicted = torch.max(X, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Test Accuracy: {accuracy:.2f}%')

Epoch 1/5, Loss: 3.2501, Test Accuracy: 30.15%
Epoch 2/5, Loss: 2.9830, Test Accuracy: 36.40%
Epoch 3/5, Loss: 2.5361, Test Accuracy: 41.41%
Epoch 4/5, Loss: 1.9488, Test Accuracy: 42.18%
Epoch 5/5, Loss: 0.9986, Test Accuracy: 43.42%


In [47]:
# guardamos el modelo
torch.save(model.state_dict(), 'models/modelo_pre_entrenado_resnet50.pth')

In [ ]:
# cargamos el modelo
# definimos el modelo
model = models.resnet50(pretrained=True)
# cambiamos la ultima capa
# sacamos el numero de features (entradas) de la ultima capa
num_ftrs = model.fc.in_features
# cambiamos la ultima capa por una nueva con 200 salidas
model.fc = nn.Linear(num_ftrs, 200)

# cargamos los pesos del modelo pre entrenado en el nuevo modelo que hemos definido arriba
model.load_state_dict(torch.load('models/modelo_pre_entrenado_resnet50.pth'))
# ponemos el modelo en modo evaluacion (no entrenamiento)
model.eval()

In [27]:
# cargamos un modelo pre entrenado de resnext50_32x4d
model_rx = models.resnext101_64x4d(pretrained=True)
model_rx

c:\Users\israe\anaconda3\envs\cuda_user\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\israe\anaconda3\envs\cuda_user\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_64X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_64X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext101_64x4d-173b62eb.pth" to C:\Users\israe/.cache\torch\hub\checkpoints\resnext101_64x4d-173b62eb.pth
100%|██████████| 319M/319M [00:54<00:00, 6.09MB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [28]:
# miramos la ultima capa del modelo pre entrenado
model_rx.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [29]:
# vemos que la salida es de 1000 neuronas, cambiamos la ultima capa por una de 200 neuronas
num_ftrs = model_rx.fc.in_features
model_rx.fc = nn.Linear(num_ftrs, 200)

# miramos la nueva ultima capa del modelo pre entrenado con 200 neuronas
model_rx.fc

Linear(in_features=2048, out_features=200, bias=True)

In [30]:
# Define a loss function and an optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # precision de un batch 17%
#optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001) # precision de un batch 19.26%
#optimizer = optim.Adamax(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001) # precision de un batch 20.64%

# Train the model on your dataset
num_epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_rx.to(device)

for epoch in range(num_epochs):
    model_rx.train()
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()
        X = model_rx(X)
        loss = criterion(X, y)
        loss.backward()
        optimizer.step()

    model_rx.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            X = model_rx(X)
            _, predicted = torch.max(X, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Test Accuracy: {accuracy:.2f}%')

Epoch 1/1, Loss: 5.2741, Test Accuracy: 0.46%


In [ ]:
# guardamos el modelo pre entrenado de resnext50_32x4d
torch.save(model_rx.state_dict(), 'models/modelo_pre_entrenado_resnext50_32x4d.pth')

In [ ]:
# cargamos el modelo pre entrenado de resnext50_32x4d
# definimos el modelo
model_rx = models.resnext50_32x4d(pretrained=True)
# cambiamos la ultima capa
# sacamos el numero de features (entradas) de la ultima capa
num_ftrs = model_rx.fc.in_features
# cambiamos la ultima capa por una nueva con 200 salidas
model_rx.fc = nn.Linear(num_ftrs, 200)

# cargamos los pesos del modelo pre entrenado en el nuevo modelo que hemos definido arriba
model_rx.load_state_dict(torch.load('models/modelo_pre_entrenado_resnext50_32x4d.pth'))
# ponemos el modelo en modo evaluacion (no entrenamiento)
model_rx.eval()